# Project Name

## Table of Contents

[Summary](#summary)

[Imports](#imports)

[Google Cloud API](#google-cloud-api)

[Spoonacular API](#spoonacular-api)

# Summary

## Functions

### General Functions
- Allow user input for adding recipes
- Make script easy to use with friendly language

### Recipe Costing
Take a recipe from a text document, calculate its recipe cost, and add it to an individual dataframe
- Read a recipe's text document in Python
- Clean data to get what's strictly necessary
- Turn recipe data into a dataframe
- Use Spoonacular API to calculate metrics
    - ~~Search for matching ingredients~~
    - ~~Get ingredient 'id'~~
    - ~~Get ingredient info~~
    - ~~Create df of ingredient info~~
    - Add ingredient info df to recipe df
- Use data extracted from Spoonacular to calculate recipe cost

### Menu Engineering
Take individual recipe data, and calculate metrics for entire menu
- Determine desired metrics for menu engineering
- Append recipe cost to Menu Engineering df
- Append recipes to menu engineering df
- Calculate menu engineering metrics

### Visualization
Create 

# Imports

In [37]:
from pathlib import Path
import csv
import pandas as pd
# import panel as pn
# pn.extension('plotly')
# import plotly.express as px
# import plotly.io as pio
# pio.renderers.default = 'iframe_connected'  # Bypass mimetype 'renderer not found'
import pandas as pd
# import hvplot.pandas
import matplotlib.pyplot as plt
# import numpy as np
# import seaborn as sns

import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv('token.env')
spoon_key = os.getenv('SPOONACULAR_KEY')
gcloud_key = os.getenv('GCLOUD_KEY')
gcloud_oauth_key = os.getenv('GCLOUD_OAUTH_KEY')
gcloud_oauth_secret = os.getenv('GCLOUD_OAUTH_SECRET')

import json, requests
from pandas.io.json import json_normalize

# import alpaca_trade_api as tradeapi

# Google Sheets API
from pprint import pprint
from googleapiclient import discovery
from oauth2client.service_account import ServiceAccountCredentials

# GSpread by burnash
import gspread

# 'chatter` functions from chatter.py
import sys
# from chatter import *

# Google Cloud API and GSpread

## Scopes and Credentials

In [ ]:
# Define scopes for Google APIs
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

# Add account credentials
creds = ServiceAccountCredentials.from_json_keyfile_name('easy-as-py-service-account.json')

# Authorize clientsheet
client = gspread.authorize(creds)

## Functions

In [ ]:
# Get instance of 'Cost Cheesecake' spreadsheet using its title
def open_spreadsheet(spreadsheet_title):
    return client.open(spreadsheet_title)

# Nested function to get specific sheet (tab) from spreadsheet instance
def open_sheet(spreadsheet_title, tab_index):
    sheet = open_spreadsheet(spreadsheet_title)
    return sheet.get_worksheet(tab_index)

# Use `for loop` to convert letter of column into number for .cell()
def letter_to_num(column_letter):
    number = ord(column_letter.lower()) - 96
    return number

def get_sheet_data(sheet_var):
    sheet_data = {sheet_var}.get_all_records()
    return sheet_data

# Get data from specified cell
def get_cell(sheet_var, column_letter, row_num):
    column_number = letter_to_num(column_letter)
    return sheet.cell(col=column_number,row=row_num)

# Turn sheet dictionary into a dataframe
def make_df(sheet_data):
    df = pd.DataFrame.from_dict(sheet_data)
    return df

# Clean dataframe by dropping rows with no value in 'Item_Name'
def clean_df(df):
    df.drop(df[df['Item_Name'] == ''].index, inplace=True)
    return df

## Data Exploration

# Spoonacular API

## Endpoints

### Base and key URLs

In [38]:
# Requests should be formatted as "api + endpoint + "?query=" + {query_params} + api_key
spoon_url = 'https://api.spoonacular.com'
key_url = '&apiKey=' + spoon_key

### Search for matching ingredients

In [39]:
# Ingredient Search endpoint
search_ing_url = spoon_url + '/food/ingredients/search?query='

# Function that takes in an ingredient name and returns up to 20 matches
# Example request: https://api.spoonacular.com/food/ingredients/search?query=banana&number=2&sort=calories&sortDirection=desc
def search_ing(ing):
    return requests.get(search_ing_url + ing + '&number=20' + key_url).json()

search_ing('all purpose flour')

{'results': [{'id': 20081, 'name': 'allpurpose flour', 'image': 'flour.png'},
  {'id': 20581, 'name': 'unbleached all purpose flour', 'image': 'flour.png'},
  {'id': 93620,
   'name': 'gluten free all purpose flour',
   'image': 'gluten-free-flour.jpg'}],
 'offset': 0,
 'number': 20,
 'totalResults': 3}

### Get ingredient 'id'

In [40]:
# Get ingredient `id`
def get_ing_id(ing):
    ingredient = search_ing(ing)['results'][0]
    print(f"Match for {ing}:")
    print(ingredient)

    ing_id = ingredient['id']
    return ing_id

get_ing_id('all purpose flour')

Match for all purpose flour:
{'id': 20081, 'name': 'allpurpose flour', 'image': 'flour.png'}


20081

### Get ingredient information

In [41]:
# Get ingredient information
def get_ing_info(ing, ing_amount, unit):
        ing_id = get_ing_id(ing)
        return requests.get(f"{spoon_url}/food/ingredients/{ing_id}/information?amount={ing_amount}&unit={unit}{key_url}").json()

get_ing_info('all purpose flour', 250, 'g')
        

Match for all purpose flour:
{'id': 20081, 'name': 'allpurpose flour', 'image': 'flour.png'}


{'id': 20081,
 'original': 'wheat flour',
 'originalName': 'wheat flour',
 'name': 'wheat flour',
 'amount': 250.0,
 'unit': 'g',
 'unitShort': 'g',
 'unitLong': 'grams',
 'possibleUnits': ['g', 'oz', 'teaspoon', 'cup', 'serving', 'tablespoon'],
 'estimatedCost': {'value': 33.33, 'unit': 'US Cents'},
 'consistency': 'solid',
 'shoppingListUnits': ['ounces'],
 'aisle': 'Baking',
 'image': 'flour.png',
 'meta': [],
 'nutrition': {'nutrients': [{'title': 'Cholesterol',
    'name': 'Cholesterol',
    'amount': 0.0,
    'unit': 'mg'},
   {'title': 'Vitamin K', 'name': 'Vitamin K', 'amount': 0.75, 'unit': 'µg'},
   {'title': 'Calories', 'name': 'Calories', 'amount': 910.0, 'unit': 'kcal'},
   {'title': 'Caffeine', 'name': 'Caffeine', 'amount': 0.0, 'unit': 'mg'},
   {'title': 'Zinc', 'name': 'Zinc', 'amount': 1.75, 'unit': 'mg'},
   {'title': 'Vitamin B12',
    'name': 'Vitamin B12',
    'amount': 0.0,
    'unit': 'µg'},
   {'title': 'Choline', 'name': 'Choline', 'amount': 26.0, 'unit': 'mg'

### Create df of ingredient info

In [45]:
# Create df from ingredient info
def ing_info_to_df(ing, ing_amount, unit):
    ing_info = get_ing_info(ing, ing_amount, unit)
    df = pd.json_normalize(ing_info)[['id','name','amount','unit','estimatedCost.value']]
    df['estimatedCost.value'] = (df['estimatedCost.value'] / 100).round(2)
    df.columns = [['ID','Ingredient_Name','Recipe_Qty','Recipe_Unit','Recipe_Cost_Per_Unit_$']]
    return df

ing_info_to_df('all purpose flour', 1, 'c')

Match for all purpose flour:
{'id': 20081, 'name': 'allpurpose flour', 'image': 'flour.png'}


,ID,Ingredient_Name,Recipe_Qty,Recipe_Unit,Recipe_Cost_Per_Unit_$
0,20081,wheat flour,1.0,c,0.17


### Add ingredient info df to recipe df

### Extract data needed for evaluation

# Final Script Flow